## Paint Scrape Project

Start by importing the necessary libraries:
Download the ones you don't have

In [1]:
import pandas as pd
import numpy as np
import json

### Sherwin Williams

We get this code generated for us by Insomnia: 
https://www.youtube.com/watch?v=DqtlR0y0suo

We start by getting the categories for Sherwin Williams' paint, but this isn't very useful.

In [2]:
import requests

url = "https://api.sherwin-williams.com/prism/v1/colors/sherwin"

querystring = {"lng":"en-US","_corev":"5.1.0"}

payload = ""
headers = {
    ## INSERT YOUR HEADERS
    }

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

r = response.json()
df = pd.json_normalize(r)
df.to_csv('sherwin_paint.csv')

In [3]:
url = "https://api.sherwin-williams.com/prism/v1/chunks/sherwin"

querystring = {"lng":"en-US","_corev":"5.1.0"}

payload = ""
headers = {
    ## INSERT YOUR HEADERS
}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

r = response.json()

In [4]:
names_df = pd.json_normalize(r['names'])

names = []
for category in names_df:
    group_number = 1
    for color_name in r['names'][category]:
        names.append((category,color_name, group_number))
        group_number += 1
        

categories = pd.DataFrame(names, columns=['Category', 'Name', 'Group'])
categories.drop_duplicates(inplace=True)


values_df = pd.json_normalize(r['values'])

# keep only categories that appear in the values
categoriesToKeep = list(values_df.columns)
categories = categories[categories['Category'].isin(categoriesToKeep)]
categories.reset_index(drop=True, inplace=True)


values = []
for category in values_df:
    name_groups = []
    group_number = 1
    for name_group in r['values'][category]:
        for value in name_group:
            values.append((category, value, group_number))
        group_number += 1
values_df = pd.DataFrame(values, columns=['Category', 'Value', 'Group'])


merged_df = pd.merge(values_df, categories, on=['Category', 'Group'])
merged_df.to_csv('sherwin_paint_categories.csv')

### Benjamin Moore

Again, using Insomnia for this cURL get

In [ ]:
url = "https://www.benjaminmoore.com/en-us/paint-colors"

payload = ""
headers = {
    ## INSERT YOUR HEADERS
}

response = requests.request("GET", url, data=payload, headers=headers)

Import the following libraries to easily parse the output

In [ ]:
from bs4 import BeautifulSoup
from itertools import chain

In [ ]:
# Opening the html file 
HTMLFile = open("benjamin.html", "r") 

soup = BeautifulSoup(HTMLFile, 'html.parser')

# Find the script tag containing __NEXT_DATA__
script_tag = soup.find('script', {'id':'__NEXT_DATA__'}).get_text()

# load that data as json to parse
jsondata = json.loads(script_tag)

r = jsondata

# Define the file path to save the JSON
file_path = 'benny.json'  # Replace with your desired file path and name

# Write the JSON object to a file with pretty formatting
with open(file_path, 'w') as file:
    json.dump(jsondata, file, indent=4)

Now we can parse the JSON file

In [ ]:
# use for categories
categories = []
for i in r['props']['pageProps']['componentData']['components']:
    if i['color_data'] is not None:
        for category in i['color_data']['data']:
            category_name = category['name']
            try:
                color_infos = category['colors']  # Replace 'my_dict' and 'key' with your dictionary and key
                for color in color_infos:
                    tupledInfo = (category_name, color['name'],color['number'],color['hex'],color['r'],color['g'],
                                  color['b'], color['description'], color['family'], color['url'], color['exterior_availability'],
                                  color['lrv'], color['wetsample_sku'], color['drysample_sku'], color['estore_available'],
                                  color['is_best_selling'], color['aliases'], color['product_types_available'],
                                  color['stainOpacitiesAvailable'], color['indexNum'], color['harmony'],
                                  color['similar'], color['equivalent'], color['palettes'], color['tags'], color['cdp_images'])
                    categories.append(tupledInfo)

            except KeyError:
                tupledInfo = ('Off White Color Collection', category['name'],category['number'],category['hex'],category['r'],category['g'],
                                  category['b'], category['description'], category['family'], category['url'], category['exterior_availability'],
                                  category['lrv'], category['wetsample_sku'], category['drysample_sku'], category['estore_available'],
                                  category['is_best_selling'], category['aliases'], category['product_types_available'],
                                  category['stainOpacitiesAvailable'], 'NA', category['harmony'],
                                  category['similar'], category['equivalent'], category['palettes'], category['tags'], category['cdp_images'])
                categories.append(tupledInfo)
                
                
tupledInfo = (
    'category_name', 'color_name', 'number', 'hex', 'r', 'g', 'b', 'description', 'family', 'url',
    'exterior_availability', 'lrv', 'wetsample_sku', 'drysample_sku', 'estore_available',
    'is_best_selling', 'aliases', 'product_types_available', 'stainOpacitiesAvailable',
    'indexNum', 'harmony', 'similar', 'equivalent', 'palettes', 'tags', 'cdp_images'
)

# Extract variable names into a list
variable_names = list(tupledInfo)


results = pd.DataFrame(categories, columns=variable_names)

results.to_csv('benjamin.csv')

### Valspar

For the valspar.html, I manually loaded the whole page to download the paint info

In [ ]:
## valspar

# Opening the html file of the Valspar url
valspar_html = open("valspar.html", "r") 

soup = BeautifulSoup(valspar_html, 'html.parser')

# Find the div tag containing 'data-cbg-cmp-hook-wall' = 'wall-item'
colors = soup.findAll('div', {'data-cbg-cmp-hook-wall':'wall-item'})

In [ ]:
# get data-color-collection, data-color-family, data-color-id, data-color-name, data-hex, data-lrv, data-sort, href,
results = []
for color in colors:
    color_id = color.get('data-color-id')
    color_name = color.get('data-color-name')
    color_category = color.get('data-color-family')
    color_collection = color.get('data-color-collection')
    color_hex = color.get('data-hex')
    color_lrv = color.get('data-lrv')
    color_link = 'valspar.com/'+color.find('a').get('href')
    
    resultTuple = (color_id, color_name, color_category, color_collection, color_hex, color_lrv, color_link)
    results.append(resultTuple)

# convert list of tuples into pd
columnNames = ['id', 'name', 'category', 'collection', 'hex', 'lrv', 'link']
valspar = pd.DataFrame(results, columns=columnNames)

# save output
valspar.to_csv('valspar.csv')

### Behr

For we need to use a Chrome driver to load the JavaScript

In [ ]:
from numpy import random
from time import sleep

#https://github.com/ultrafunkamsterdam/undetected-chromedriver
import undetected_chromedriver as uc
driver = uc.Chrome(headless=True, use_subprocess=False)

We also need to use a YAML extractor, which is made easy through the Chomre extension selectorlib

In [ ]:
from selectorlib import Extractor

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('Behr.yml')

Now we can similarly use Insomnia to make a request to get all the colors that Behr has

In [2]:
url = "https://www.behr.com/omsproductservice/services/getAllAvailableColorIds"

querystring = {"prodcode":"cf","surface":""}

payload = ""
headers = {
    ## insert header
}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

r = response.json()

allUrls = []
for i in r:
    currentURL = 'https://www.behr.com/consumer/ColorDetailView/' + i
    allUrls.append(currentURL)

Go through all the urls for the product and save the content to a csv file

In [9]:
allResults = []
driver = uc.Chrome(headless=True, use_subprocess=False)
x = 1

for url in allUrls:
    driver.get(url)
    sleep(2)
    html_content = driver.page_source
    data = e.extract(html_content)
    allResults.append(data)
    x +=1
driver.close()

# using these column names create a pandas datagrame
columnNames = ['Name', 'mfg_num', 'lrv', 'rgb', 'category', 'hexcode']
results = pd.DataFrame(allResults, columns=columnNames)


results.to_csv('behr.csv')